In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import backend
import tensorflow
tensorflow.set_random_seed(7)
from sklearn.model_selection import train_test_split
import numpy
numpy.random.seed(7)

Using TensorFlow backend.


## Restricting GPU memory usage

The code here should be added to any work you do on Volta.  If you don't, then your code will monopolize all available memory on each of the 4 GPUs on the machine, preventing others from working on it.  If you do **that**, you will be frowned upon.

The code in the next cell has the effect that:
1. Memory use will start off with some small fraction of the memory on each GPU.
1. It will grow if necessary (since `allow_growth` is set to `True`).
1. It will max out at 5% of overall memory.  Given the GPUs we have, this gives you (4 x 808 MB), which should be sufficient here.

In [4]:
########################
# Limit TensorFlow GPU use.
config = tensorflow.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.05
backend.tensorflow_backend.set_session(tensorflow.Session(config=config))
########################

## Loading our data

The function `train_test_split` comes from `sklearn` (from `SciKit`). Two things to note:
1. Setting `test_size` to `0.25` means that 25% of the data is held aside as testing data.  
1. Randomness here can affect overall results.

In [60]:
dataset = numpy.loadtxt("diabetes.csv", delimiter=",")

# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.25)

## Building and evaluating our model

The initial model is a simple network, with two hidden layers, each of which uses the RELU function for activation.  There is also a simple single-class output neuron using the Sigmoid logistic function.  The loss function is binary cross entropy (and so we want to measure the associated binary accuracy value).  The model is trained using a batch size equal to all of the training data, for 1,000 epochs.

For this network, do the following:
1. Explain what binary cross entropy function is, exactly, and why it is used.   
**Answer:** Cross-entropy loss measures the performance of a classification model whose output is a probability value between 0 and 1. In this network, we need to evaluate patients have diabetes or not. The predict value should between 0 and 1. And cross-entropy loss increases as the predicted probability diverges from the actual label(1 or 0). So it is a good solution to use this loss function for back propagation.   
2. Try increasing the number of epochs to larger values, such as 2,000, 5,000, etc.  What are the effects of this on the accuracy results you get afterwards?  Why?  (Remember that you have to wait until the `model.fit()` function call is complete before executing the following cells, or else the results you get will be inconsistent.  
**Answer:** After I increase the epoch from 1000 to 2000 or 5000. The accuracy of training data go up but the accuracy of testing data fall down. I think the reason is the model learned a lot of unnecessary features of this data and get overfitting. So this could make this model could fit well in training data but work poor in testing data. 
3. Return the epochs to 1,000.  Examine the effects of adding two dropout layers, one after each of the 25-neuron Dense layers.  Try different dropour parameters and see what happens.  Explain what you saw. A useful guide and discussion can be found [at the following post](https://machinelearningmastery.com/dropout-regularization-deep-learning-models-keras/).   
**Answer:** Dropout is a efficient solution to avoid overfitting. When I set a dropout layer with drop rate 0.2, accuracy of testing data and training data are closest. When the drop rate is close to 1, the effects will be reduced.

In [76]:
# create model
model = Sequential()
model.add(Dense(25, input_dim=8, activation='relu'))
model.add(Dropout(0.2, input_shape=(8,)))
model.add(Dense(25, activation='relu'))
model.add(Dropout(0.2, input_shape=(8,)))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
# Fit the model
model.fit(train_x, train_y, epochs=1000, batch_size=len(train_x), validation_data=(test_x, test_y), verbose=1)

Train on 576 samples, validate on 192 samples
Epoch 1/1000
576/576 [==============================] - 1s 2ms/step - loss: 9.6608 - binary_accuracy: 0.3559 - val_loss: 10.1140 - val_binary_accuracy: 0.3646
Epoch 2/1000
576/576 [==============================] - 0s 13us/step - loss: 9.3868 - binary_accuracy: 0.3698 - val_loss: 10.1033 - val_binary_accuracy: 0.3646
Epoch 3/1000
576/576 [==============================] - 0s 14us/step - loss: 9.1559 - binary_accuracy: 0.3646 - val_loss: 10.0920 - val_binary_accuracy: 0.3646
Epoch 4/1000
576/576 [==============================] - 0s 12us/step - loss: 8.9534 - binary_accuracy: 0.3837 - val_loss: 10.0804 - val_binary_accuracy: 0.3646
Epoch 5/1000
576/576 [==============================] - 0s 13us/step - loss: 9.1481 - binary_accuracy: 0.3785 - val_loss: 10.0544 - val_binary_accuracy: 0.3646
Epoch 6/1000
576/576 [==============================] - 0s 12us/step - loss: 8.9889 - binary_accuracy: 0.3767 - val_loss: 10.0138 - val_binary_accuracy: 0.

Epoch 52/1000
576/576 [==============================] - 0s 13us/step - loss: 5.5810 - binary_accuracy: 0.6267 - val_loss: 5.7944 - val_binary_accuracy: 0.6406
Epoch 53/1000
576/576 [==============================] - 0s 13us/step - loss: 5.6893 - binary_accuracy: 0.6233 - val_loss: 5.7926 - val_binary_accuracy: 0.6406
Epoch 54/1000
576/576 [==============================] - 0s 13us/step - loss: 5.4777 - binary_accuracy: 0.6424 - val_loss: 5.7925 - val_binary_accuracy: 0.6406
Epoch 55/1000
576/576 [==============================] - 0s 13us/step - loss: 5.5578 - binary_accuracy: 0.6285 - val_loss: 5.7925 - val_binary_accuracy: 0.6406
Epoch 56/1000
576/576 [==============================] - 0s 14us/step - loss: 5.4547 - binary_accuracy: 0.6406 - val_loss: 5.7925 - val_binary_accuracy: 0.6406
Epoch 57/1000
576/576 [==============================] - 0s 14us/step - loss: 5.3884 - binary_accuracy: 0.6406 - val_loss: 5.7926 - val_binary_accuracy: 0.6406
Epoch 58/1000
576/576 [=================

Epoch 104/1000
576/576 [==============================] - 0s 12us/step - loss: 5.5498 - binary_accuracy: 0.6111 - val_loss: 5.5814 - val_binary_accuracy: 0.6354
Epoch 105/1000
576/576 [==============================] - 0s 13us/step - loss: 5.0220 - binary_accuracy: 0.6580 - val_loss: 5.5796 - val_binary_accuracy: 0.6354
Epoch 106/1000
576/576 [==============================] - 0s 12us/step - loss: 5.3019 - binary_accuracy: 0.6319 - val_loss: 5.5779 - val_binary_accuracy: 0.6354
Epoch 107/1000
576/576 [==============================] - 0s 12us/step - loss: 5.2932 - binary_accuracy: 0.6337 - val_loss: 5.5743 - val_binary_accuracy: 0.6354
Epoch 108/1000
576/576 [==============================] - 0s 13us/step - loss: 5.1946 - binary_accuracy: 0.6354 - val_loss: 5.5705 - val_binary_accuracy: 0.6354
Epoch 109/1000
576/576 [==============================] - 0s 13us/step - loss: 5.3225 - binary_accuracy: 0.6389 - val_loss: 5.5647 - val_binary_accuracy: 0.6354
Epoch 110/1000
576/576 [==========

Epoch 155/1000
576/576 [==============================] - 0s 15us/step - loss: 3.7928 - binary_accuracy: 0.6354 - val_loss: 3.3280 - val_binary_accuracy: 0.6198
Epoch 156/1000
576/576 [==============================] - 0s 16us/step - loss: 4.3521 - binary_accuracy: 0.5764 - val_loss: 3.2173 - val_binary_accuracy: 0.6198
Epoch 157/1000
576/576 [==============================] - 0s 14us/step - loss: 4.1717 - binary_accuracy: 0.6128 - val_loss: 3.1095 - val_binary_accuracy: 0.6250
Epoch 158/1000
576/576 [==============================] - 0s 14us/step - loss: 4.4245 - binary_accuracy: 0.5677 - val_loss: 3.0041 - val_binary_accuracy: 0.6250
Epoch 159/1000
576/576 [==============================] - 0s 16us/step - loss: 4.0090 - binary_accuracy: 0.6215 - val_loss: 2.9083 - val_binary_accuracy: 0.6198
Epoch 160/1000
576/576 [==============================] - 0s 15us/step - loss: 4.1024 - binary_accuracy: 0.6042 - val_loss: 2.7405 - val_binary_accuracy: 0.6406
Epoch 161/1000
576/576 [==========

Epoch 206/1000
576/576 [==============================] - 0s 11us/step - loss: 1.5592 - binary_accuracy: 0.5781 - val_loss: 0.6737 - val_binary_accuracy: 0.6458
Epoch 207/1000
576/576 [==============================] - 0s 11us/step - loss: 1.3751 - binary_accuracy: 0.5972 - val_loss: 0.6673 - val_binary_accuracy: 0.6510
Epoch 208/1000
576/576 [==============================] - 0s 15us/step - loss: 1.4977 - binary_accuracy: 0.5851 - val_loss: 0.6635 - val_binary_accuracy: 0.6562
Epoch 209/1000
576/576 [==============================] - 0s 13us/step - loss: 1.4542 - binary_accuracy: 0.5642 - val_loss: 0.6633 - val_binary_accuracy: 0.6615
Epoch 210/1000
576/576 [==============================] - 0s 13us/step - loss: 1.5238 - binary_accuracy: 0.5694 - val_loss: 0.6657 - val_binary_accuracy: 0.6615
Epoch 211/1000
576/576 [==============================] - 0s 11us/step - loss: 1.4603 - binary_accuracy: 0.5816 - val_loss: 0.6714 - val_binary_accuracy: 0.6354
Epoch 212/1000
576/576 [==========

...

Epoch 767/1000
576/576 [==============================] - 0s 13us/step - loss: 0.6336 - binary_accuracy: 0.6597 - val_loss: 0.6143 - val_binary_accuracy: 0.6458
Epoch 768/1000
576/576 [==============================] - 0s 13us/step - loss: 0.6260 - binary_accuracy: 0.6562 - val_loss: 0.6149 - val_binary_accuracy: 0.6458
Epoch 769/1000
576/576 [==============================] - 0s 12us/step - loss: 0.6265 - binary_accuracy: 0.6649 - val_loss: 0.6159 - val_binary_accuracy: 0.6406
Epoch 770/1000
576/576 [==============================] - 0s 12us/step - loss: 0.6244 - binary_accuracy: 0.6632 - val_loss: 0.6168 - val_binary_accuracy: 0.6458
Epoch 771/1000
576/576 [==============================] - 0s 13us/step - loss: 0.6207 - binary_accuracy: 0.6719 - val_loss: 0.6175 - val_binary_accuracy: 0.6510
Epoch 772/1000
576/576 [==============================] - 0s 15us/step - loss: 0.6426 - binary_accuracy: 0.6476 - val_loss: 0.6185 - val_binary_accuracy: 0.6562
Epoch 773/1000
576/576 [==========

Epoch 818/1000
576/576 [==============================] - 0s 14us/step - loss: 0.6425 - binary_accuracy: 0.6736 - val_loss: 0.6186 - val_binary_accuracy: 0.6406
Epoch 819/1000
576/576 [==============================] - 0s 13us/step - loss: 0.6203 - binary_accuracy: 0.6736 - val_loss: 0.6188 - val_binary_accuracy: 0.6458
Epoch 820/1000
576/576 [==============================] - 0s 13us/step - loss: 0.5994 - binary_accuracy: 0.6840 - val_loss: 0.6189 - val_binary_accuracy: 0.6458
Epoch 821/1000
576/576 [==============================] - 0s 13us/step - loss: 0.5988 - binary_accuracy: 0.6962 - val_loss: 0.6188 - val_binary_accuracy: 0.6458
Epoch 822/1000
576/576 [==============================] - 0s 13us/step - loss: 0.6450 - binary_accuracy: 0.6701 - val_loss: 0.6186 - val_binary_accuracy: 0.6458
Epoch 823/1000
576/576 [==============================] - 0s 15us/step - loss: 0.6199 - binary_accuracy: 0.6667 - val_loss: 0.6183 - val_binary_accuracy: 0.6458
Epoch 824/1000
576/576 [==========

Epoch 869/1000
576/576 [==============================] - 0s 11us/step - loss: 0.6104 - binary_accuracy: 0.7049 - val_loss: 0.6145 - val_binary_accuracy: 0.6562
Epoch 870/1000
576/576 [==============================] - 0s 13us/step - loss: 0.6100 - binary_accuracy: 0.6823 - val_loss: 0.6138 - val_binary_accuracy: 0.6615
Epoch 871/1000
576/576 [==============================] - 0s 12us/step - loss: 0.6174 - binary_accuracy: 0.6875 - val_loss: 0.6134 - val_binary_accuracy: 0.6615
Epoch 872/1000
576/576 [==============================] - 0s 11us/step - loss: 0.5975 - binary_accuracy: 0.7014 - val_loss: 0.6130 - val_binary_accuracy: 0.6615
Epoch 873/1000
576/576 [==============================] - 0s 13us/step - loss: 0.6160 - binary_accuracy: 0.6649 - val_loss: 0.6127 - val_binary_accuracy: 0.6510
Epoch 874/1000
576/576 [==============================] - 0s 13us/step - loss: 0.6303 - binary_accuracy: 0.6910 - val_loss: 0.6126 - val_binary_accuracy: 0.6458
Epoch 875/1000
576/576 [==========

Epoch 920/1000
576/576 [==============================] - 0s 12us/step - loss: 0.6159 - binary_accuracy: 0.6771 - val_loss: 0.6079 - val_binary_accuracy: 0.6510
Epoch 921/1000
576/576 [==============================] - 0s 13us/step - loss: 0.6087 - binary_accuracy: 0.6858 - val_loss: 0.6079 - val_binary_accuracy: 0.6510
Epoch 922/1000
576/576 [==============================] - 0s 13us/step - loss: 0.6147 - binary_accuracy: 0.6944 - val_loss: 0.6080 - val_binary_accuracy: 0.6510
Epoch 923/1000
576/576 [==============================] - 0s 16us/step - loss: 0.6119 - binary_accuracy: 0.6858 - val_loss: 0.6083 - val_binary_accuracy: 0.6562
Epoch 924/1000
576/576 [==============================] - 0s 13us/step - loss: 0.6086 - binary_accuracy: 0.6840 - val_loss: 0.6085 - val_binary_accuracy: 0.6510
Epoch 925/1000
576/576 [==============================] - 0s 14us/step - loss: 0.6021 - binary_accuracy: 0.6927 - val_loss: 0.6086 - val_binary_accuracy: 0.6510
Epoch 926/1000
576/576 [==========

Epoch 971/1000
576/576 [==============================] - 0s 15us/step - loss: 0.6054 - binary_accuracy: 0.6875 - val_loss: 0.6073 - val_binary_accuracy: 0.6667
Epoch 972/1000
576/576 [==============================] - 0s 11us/step - loss: 0.6136 - binary_accuracy: 0.6823 - val_loss: 0.6075 - val_binary_accuracy: 0.6667
Epoch 973/1000
576/576 [==============================] - 0s 12us/step - loss: 0.6088 - binary_accuracy: 0.7049 - val_loss: 0.6076 - val_binary_accuracy: 0.6615
Epoch 974/1000
576/576 [==============================] - 0s 12us/step - loss: 0.6003 - binary_accuracy: 0.6840 - val_loss: 0.6077 - val_binary_accuracy: 0.6615
Epoch 975/1000
576/576 [==============================] - 0s 13us/step - loss: 0.6145 - binary_accuracy: 0.6736 - val_loss: 0.6077 - val_binary_accuracy: 0.6615
Epoch 976/1000
576/576 [==============================] - 0s 12us/step - loss: 0.6036 - binary_accuracy: 0.6771 - val_loss: 0.6080 - val_binary_accuracy: 0.6562
Epoch 977/1000
576/576 [==========

## Evaluation of final model on training data

In [74]:
# evaluate the model
scores = model.evaluate(train_x, train_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

576/576 [==============================] - 0s 67us/step

binary_accuracy: 71.88%


## Evaluation of final model on testing data

In [75]:
# evaluate the model
scores = model.evaluate(test_x, test_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

192/192 [==============================] - 0s 65us/step

binary_accuracy: 71.88%
